In [ ]:
import torch

In [95]:
%cd /export/data4/vzhekova/biases-data

/export/data4/vzhekova/biases-data


In [96]:
!echo -e "\nFirst lines of English:\n"
!head data.en-fr.en
!echo -e "\nFirst lines of French:\n"
!head data.en-fr.fr


First lines of English:

﻿Now, I thought, "How could I really capture this?
I mean, from this entry, it would seem that I was born into a world that perceived someone like me to have nothing positive whatsoever going for them, when in fact, today I'm celebrated for the opportunities and adventures my life has procured.
So, I immediately went to look up the 2009 online edition, expecting to find a revision worth noting.
His name was Dr. Pizzutillo, an Italian American, whose name, apparently, was too difficult for most Americans to pronounce, so he went by Dr. P. And Dr. P always wore really colorful bow ties and had the very perfect disposition to work with children.
And, one day, he came in to my session — exhaustive and unforgiving, these sessions — and he said to me, "Wow.
Now, of course, this was a simple ploy on Dr. P's part to get me to do the exercises I didn't want to do before the prospect of being the richest five-year-old in the second floor ward, but what he effectively di

In [134]:
# Tokenize text; MuST-SHE dataset
# TODO: write script for preprocessing
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer, MosesDetokenizer
from __future__ import print_function

mpn = MosesPunctNormalizer()
mt_en = MosesTokenizer(lang='en')
md_en = MosesDetokenizer(lang='en')

with open('data.en-fr.en') as fin, open('data.en-fr.tok.en','w') as fout:
    for line in fin:
        tokens = mt_en.tokenize(md_en.detokenize(mpn.normalize(line)), return_str=True)
        print(tokens, end='\n', file=fout) 
        
mt_fr = MosesTokenizer(lang='fr')
md_fr = MosesDetokenizer(lang='fr')

with open('data.en-fr.fr') as fin, open('data.en-fr.tok.fr','w') as fout:
    for line in fin:
        tokens = mt_fr.tokenize(md_fr.detokenize(mpn.normalize(line)), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished tokenizing.')

Finished tokenizing.


In [140]:
# Detokenize text

# mpn = MosesPunctNormalizer()
# mt = MosesTokenizer(lang='en')
# md = MosesDetokenizer(lang='en')

# md.detokenize(mt.tokenize(md.detokenize(mpn.normalize('So, I immediately went to look up the 2009 online edition, expecting to find a revision worth noting.').split())))

with open('data.en-fr.tok.en') as fin, open('data.en-fr.detok.en','w') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout) 

with open('data.en-fr.tok.fr') as fin, open('data.en-fr.detok.fr','w') as fout:
    for line in fin:
        tokens = md_fr.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


In [218]:
# Dividing tokenized text into subword units

!subword-nmt learn-bpe -s 32000 < data.en-fr.tok.en > sw.model.en
!subword-nmt apply-bpe -c sw.model.en < data.en-fr.tok.en > sw.data.en

!subword-nmt learn-bpe -s 32000 < data.en-fr.tok.fr > sw.model.fr
!subword-nmt apply-bpe -c sw.model.fr < data.en-fr.tok.fr > sw.data.fr

print('Finished subword.')

 16%|#####7                               | 5014/32000 [00:10<00:56, 475.25it/s]
Finished subword.


In [219]:
!echo -e "\nFirst lines of tokenized English:\n"
!head sw.data.en

!echo -e "\nFirst lines of tokenized French:\n"
!head sw.data.fr


First lines of tokenized English:

﻿ Now , I thought , &quot; How could I really capture this ?
I mean , from this ent@@ ry , it would seem that I was born into a world that percei@@ ved someone like me to have nothing positive whatsoever going for them , when in fact , today I &apos;m celebr@@ ated for the oppor@@ t@@ uni@@ ties and adventures my life has pro@@ cur@@ ed .
So , I immediately went to look up the 200@@ 9 online e@@ dition , expec@@ ting to find a re@@ vision wor@@ th n@@ oting .
His name was Dr. P@@ i@@ zz@@ u@@ til@@ lo , an Italian American , whose name , apparently , was too difficult for most Americ@@ ans to pr@@ on@@ ounce , so he went by Dr. P@@ . And Dr. P always wor@@ e really colorful bow ties and had the very perfect disp@@ os@@ i@@ tion to work with children .
And , one day , he came in to my session - exha@@ us@@ tive and un@@ for@@ giving , these sessions - and he said to me , &quot; W@@ ow .
Now , of course , this was a simple p@@ loy on Dr. P &apos;s part

In [228]:
# Binarize the data for training; test with moses tokenizer and subword_nmt failed -> BLEU score of 4

# reuse model dict; solution for dictionary size to match test dataset
# map words appearing less than threshold times to unknown 
# --tokenizer moses \
# --bpe subword_nmt \
!fairseq-preprocess \
    --source-lang en \
    --target-lang fr \
    --testpref sw.data.en-fr \
    --srcdict wmt14.en-fr.joined-dict.transformer/dict.en.txt \
    --tgtdict wmt14.en-fr.joined-dict.transformer/dict.fr.txt \
    --destdir data-bin_en-fr \
    --thresholdtgt 0 \
    --thresholdsrc 0 \
    --workers 8

2023-01-30 15:44:45 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='wmt14.en-

In [229]:
%cd /export/data4/vzhekova/biases-data

/export/data4/vzhekova/biases-data


In [232]:
# Generate translations
!fairseq-generate data-bin_en-fr  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path wmt14.en-fr.joined-dict.transformer/model.pt \
    --beam 5 \
    --batch-size 256 \
    --memory-efficient-fp16 \
    --remove-bpe=subword_nmt > en-fr.decode.log

2023-01-30 15:56:35 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': 

In [233]:
# Extract the hypotheses and references from the decoding log file
!grep ^H en-fr.decode.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.en-fr.txt
!grep ^T en-fr.decode.log | sed 's/^T-//g' | cut -f 2 | sed 's/ @@//g' > ./ref.en-fr.txt

!head ./hyp.en-fr.txt
print("..........")
!head ./ref.en-fr.txt

Je voulais les ouvrir en tant qu&apos; alliés de la communauté de la LGBTQ .
Au début de mon séjour là @-@ bas , en 2000 , je m&apos; intéressais beaucoup aux collectivités .
Après ça , j &quot; étais un peu impatient d&apos; aller à mon prochain rendez @-@ vous zéros .
Je suis allé à la maison pour célébrer , pour la conquérir .
Et je pensais que c &quot; était super cool , alors je le montrais à mon ami .
Je m&apos; appelle Dan , je suis partenaire d&apos; une conquête créative mondiale .
Je suis retourné à mon laboratoire de Boston et j&apos; ai mené une petite expérience .
Et le juge l&apos; a reconnu comme un adulte , mais je vois cela .
Nous avons donc ici une jeune femme et un homme engagés dans un jeu d&apos; aumônerie .
J&apos; ai eu le grand privilège , lorsque j &quot; étais jeune avocat , de faire la connaissance de &quot; Parks &quot; .
..........
Je <<unk>> ouvrir ces portes en tant qu&apos; alliée de la communauté LGBTQ .
À mes débuts en 2000 , j&apos; étais très intéres

In [234]:
# Detokenize text        

with open('hyp.en-fr.txt', encoding='utf8') as fin, open('hyp_detok.en-fr.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_en.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)
        
with open('ref.en-fr.txt', encoding='utf8') as fin, open('ref_detok.en-fr.txt','w', encoding='utf8') as fout:
    for line in fin:
        tokens = md_fr.detokenize(line.split(), return_str=True)
        print(tokens, end='\n', file=fout)

print('Finished detokenizing.')

Finished detokenizing.


In [235]:
!head ./hyp_detok.en-fr.txt
print("..........")
!head ./ref_detok.en-fr.txt

Je voulais les ouvrir en tant qu' alliés de la communauté de la LGBTQ.
Au début de mon séjour là-bas, en 2000, je m' intéressais beaucoup aux collectivités.
Après ça, j "étais un peu impatient d' aller à mon prochain rendez-vous zéros.
Je suis allé à la maison pour célébrer, pour la conquérir.
Et je pensais que c "était super cool, alors je le montrais à mon ami.
Je m' appelle Dan, je suis partenaire d' une conquête créative mondiale.
Je suis retourné à mon laboratoire de Boston et j' ai mené une petite expérience.
Et le juge l' a reconnu comme un adulte, mais je vois cela.
Nous avons donc ici une jeune femme et un homme engagés dans un jeu d' aumônerie.
J' ai eu le grand privilège, lorsque j "étais jeune avocat, de faire la connaissance de" Parks ".
..........
Je <<unk>> ouvrir ces portes en tant qu'alliée de la communauté LGBTQ.
À mes débuts en 2000, j'étais très intéressé par les communautés.
Après lui, j'étais un peu <<unk>> pour mon prochain <<unk>> zéro.
Je suis allée chez <<unk>

In [236]:
# Evaluate the model
# BLEU score of 28.9 (beam=5), BLEU score of 28.2 (beam=1) before detokenization
# BLEU score of 18.7 (beam=5), BLEU score of 18.2 (beam=1) after detokenization
!cat ./hyp_detok.en-fr.txt | sacrebleu ./ref_detok.en-fr.txt

{
 "name": "BLEU",
 "score": 18.7,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "50.3/27.7/17.0/10.5 (BP = 0.839 ratio = 0.851 hyp_len = 36079 ref_len = 42405)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [239]:
!fairseq-score \
    --sacrebleu \
    --sys ./hyp_detok.en-fr.txt \
    --ref ./ref_detok.en-fr.txt

Namespace(ignore_case=False, order=4, ref='./ref_detok.en-fr.txt', sacrebleu=True, sentence_bleu=False, sys='./hyp_detok.en-fr.txt')
Traceback (most recent call last):
  File "/home/vzhekova/miniconda3/envs/nmt/bin/fairseq-score", line 8, in <module>
    sys.exit(cli_main())
  File "/home/vzhekova/fairseq/fairseq_cli/score.py", line 98, in cli_main
    score(f)
  File "/home/vzhekova/fairseq/fairseq_cli/score.py", line 61, in score
    print(sacrebleu.corpus_bleu(fdsys, [fdref]).format())
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/compat.py", line 37, in corpus_bleu
    return metric.corpus_score(hypotheses, references)
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/metrics/base.py", line 414, in corpus_score
    self._check_corpus_score_args(hypotheses, references)
  File "/home/vzhekova/miniconda3/envs/nmt/lib/python3.8/site-packages/sacrebleu/metrics/base.py", line 258, in _check_corpus_score_args
    raise Type

In [237]:
# Evaluate gender accuracy; 0.46
!python ./mustshe_acc_v1.1.py \
    --input hyp.en-fr.txt \
    --tsv-definition /export/data4/vzhekova/MuST-SHE_v1.2/MuST-SHE-v1.2-data/tsv/MONOLINGUAL.fr_v1.2.tsv

Category	Term Coverage	Gender Accuracy
-------------------------------------------------
1F	0.03773584905660377	0.16666666666666666
1M	0.07804878048780488	0.5777777777777777
2F	0.12447257383966245	0.3787878787878788
2M	0.1203883495145631	0.5405405405405406
3F	0.0	0.0
3M	0.0	0.0
4F	0.07407407407407407	0.0
4M	0.16666666666666666	0.5
-------------------------------------------------
Global	0.09391395592864638	0.4604651162790698
-------------------------------------------------
